In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import method_function_adjust as mf
from tqdm import tqdm
from sklearn.metrics import r2_score
import statsmodels.api as sm

# suppress all warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
#load data
feature = np.load("..//np_files//feature.npy")
label = np.load("..//np_files//label.npy")
feature_with_constant = np.ones((feature.shape[0],feature.shape[1],feature.shape[2]+1))
feature_with_constant[:,:,1:] = feature
feature_with_constant.shape

(1115, 629, 8)

In [ ]:
#rolling validation    
#N = 20
for N in [15]:
    cost = [] # store the out-of-sample cost
    decision = [] # store the out-of-sample decision
    true_decision = [] # store the optimal decision
    alpha_list = [] # store the alpha
    time = [] # store the running time
    cost_name = ['decentralised_ols','shrunken_saa','Random Forest',
                'shrunken_non_linear','DAC','PAB_linear',
                'PAB_tree','centralised_ols','shrunken_ols']
    for inital_point in tqdm(range(0,label.shape[1] - N )):
        # print(inital_point)
        test_point = inital_point + N
        
        # obtain valid historical data and test data
        X_hats = []
        y_hats = []
        X_test = []
        y_test = []
        X_PAB = feature_with_constant[:,inital_point:inital_point + N,:]
        y_PAB = label[:,inital_point:inital_point + N]
        for k in range(label.shape[0]):
            X_temp = []
            y_temp = []
            X_test.append(feature_with_constant[k,test_point])
            y_test.append(label[k,test_point])
            for j in range(inital_point,inital_point + N):
                if label[k,j] != 0:
                    y_temp.append(label[k,j])
                    X_temp.append(feature_with_constant[k,j])
            X_hats.append(np.array(X_temp))
            y_hats.append(np.array(y_temp))

        true_decision.append(y_test)
        # obtain the out-of-sample cost
        cost_temp,decision_temp,time_temp,alpha = mf.main(X_hats,y_hats, X_test,y_test,X_PAB,y_PAB)
        cost.append(cost_temp)
        decision.append(decision_temp)
        alpha_list.append(alpha)
        time.append(time_temp)
    cost = np.array(cost)
    decision = np.array(decision)
    time = np.array(time)
    alpha_list = np.array(alpha_list)